<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Les-imports-et-configuration-du-document" data-toc-modified-id="Les-imports-et-configuration-du-document-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Les imports et configuration du document</a></span><ul class="toc-item"><li><span><a href="#La-lecture-des-fichiers-d'un-répértoire" data-toc-modified-id="La-lecture-des-fichiers-d'un-répértoire-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La lecture des fichiers d'un répértoire</a></span></li></ul></li></ul></div>

# Les imports et configuration du document

In [1]:
import pandas as pd 
import seaborn as sns
import warnings 
import os
from datetime import datetime
warnings.filterwarnings(action="ignore")

%pylab inline

plt.style.use('seaborn-darkgrid')
sns.set(font_scale=3)

os.chdir("../donnees")

Populating the interactive namespace from numpy and matplotlib


In [2]:
!dir

 Le volume dans le lecteur C s'appelle OS_Install
 Le num‚ro de s‚rie du volume est F4F1-3800

 R‚pertoire de C:\Formation\donnees

30/11/2021  14:06    <DIR>          .
30/11/2021  14:06    <DIR>          ..
29/01/2018  11:42             2ÿ436 50_Startups.csv
26/02/2018  05:29           240ÿ949 abalone.csv
21/02/2018  01:44         1ÿ171ÿ439 agents.json
20/11/2016  12:12           785ÿ065 AirQualityUCI01.csv
03/09/2015  14:39           785ÿ065 AirQualityUCI02.csv
20/11/2016  12:14           785ÿ065 AirQualityUCI03.csv
20/11/2016  12:34           783ÿ119 AirQualityUCI04.csv
22/07/2018  08:35            49ÿ082 boston.housing.data.txt
26/02/2018  05:30           129ÿ652 breastCancer.csv
10/05/2018  14:56         1ÿ423ÿ529 California Housing Price.csv
26/02/2018  14:33       152ÿ996ÿ104 creditCard.csv
25/04/2019  08:16           315ÿ987 creditDesc.csv
07/10/2021  14:37         1ÿ010ÿ610 Data_Cortex_Nuclear.csv
16/09/2021  14:04            23ÿ873 diabetes.csv
13/03/2017  04:25        76ÿ77

## La lecture des fichiers d'un répértoire 

In [3]:
def dateparse(d):
    return pd.datetime.strptime(d,'%Y%m%d%H%M%S')

meteo = pd.DataFrame()

for dirname, _, filenames in os.walk('meteo'):
    for filename in filenames:
        if filename.rfind('synop') >= 0 :
            meteo = pd.concat([meteo,
                               pd.read_csv(os.path.join(dirname, filename),
                                           sep=';',
                                           usecols=['numer_sta','date','dd','ff','t','u','vv',
                                                    'pres','rr1','rr3','rr6','rr12','rr24'],
                                           na_values='mq',
                                           dtype={'numer_sta':str,'date':str},
                                           parse_dates={'DateHeure': ['date']},
                                           date_parser=dateparse
                                          ).rename( columns={'numer_sta':'Station',
                                                              'date':'Date',
                                                              'dd':'DirectionVent',
                                                              'ff':'VitesseVent',
                                                              't':'Temperature',
                                                              'u':'Humidite',
                                                              'vv':'Visibilite',
                                                              'pres':'Pression',
                                                              'rr1' :'Precipitation01',
                                                              'rr3' :'Precipitation03',
                                                              'rr6' :'Precipitation06',
                                                              'rr12':'Precipitation12',
                                                              'rr24':'Precipitation24'})],axis=0)
            
meteo.Temperature = meteo.Temperature - 273.15
meteo.Pression    = meteo.Pression / 100
meteo.Visibilite  = meteo.Visibilite / 1000
meteo.head(5)            

,DateHeure,Station,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Precipitation01,Precipitation03,Precipitation06,Precipitation12,Precipitation24
0,2020-01-01,07005,120.0,1.8,1.2,89.0,9.29,1022.6,0.0,0.0,0.0,0.0,-0.1
1,2020-01-01,07015,80.0,4.7,2.1,99.0,0.13,1027.2,0.0,0.0,0.0,-0.1,-0.1
2,2020-01-01,07020,80.0,1.3,7.4,93.0,8.00,1027.6,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,07027,100.0,4.2,2.6,100.0,0.35,1022.3,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,07037,130.0,2.2,-0.9,100.0,0.08,1012.5,0.0,0.0,0.0,0.0,0.0


In [4]:
meteo.shape

(174227, 13)

In [5]:
postes = pd.read_csv('meteo/postesSynop.csv',sep=';',dtype={'ID':str})
postes.Nom =postes.Nom.apply(lambda x : x if x in ['CLERMONT-FD','MONT-DE-MARSAN',
                                       'ST-PIERRE','ST-BARTHELEMY METEO'] 
                               else x[0:x.find('-')] 
                                    if x.find('-') != -1 else x).apply(lambda x : str(x).title())
postes.Altitude = postes.Altitude.astype('int16')
postes = postes[postes.ID < '08000']
meteo = postes.merge(meteo, how = "inner", 
                     left_on = "ID", right_on = "Station").drop(["ID","Station"], axis = "columns")

In [6]:
meteo['Precipitation'] =  meteo['Precipitation03'].combine_first(meteo['Precipitation06']/2)\
                                                  .combine_first(meteo['Precipitation12']/4)\
                                                  .combine_first(meteo['Precipitation24']/8)\
                                                  .combine_first(meteo['Precipitation01']*3)

In [7]:
meteo.drop(columns=['Precipitation06',
                    'Precipitation12',
                    'Precipitation24',
                    'Precipitation01',
                    'Precipitation03'], inplace=True)

In [8]:
meteo.to_parquet('meteo.gzip',compression='gzip', engine='pyarrow')

In [9]:
meteo.to_csv('meteo.csv', index=None, index_label=None)

In [10]:
%%time
pd.read_csv('meteo.csv').dtypes

Wall time: 299 ms


Nom               object
Latitude         float64
Longitude        float64
Altitude           int64
DateHeure         object
DirectionVent    float64
VitesseVent      float64
Temperature      float64
Humidite         float64
Visibilite       float64
Pression         float64
Precipitation    float64
dtype: object

In [11]:
%%time
meteo = pd.read_parquet('meteo.gzip', engine='pyarrow')
meteo.dtypes

Wall time: 63.8 ms


Nom                      object
Latitude                float64
Longitude               float64
Altitude                  int16
DateHeure        datetime64[ns]
DirectionVent           float64
VitesseVent             float64
Temperature             float64
Humidite                float64
Visibilite              float64
Pression                float64
Precipitation           float64
dtype: object